In [ ]:
#default_exp loss

In [ ]:
#export
from fastai2.basics import *
from fastai2.callback.all import *
from torchvision.models import vgg16, vgg19

In [ ]:
#export
def gram(x):
    n,c,h,w = x.size()
    x = x.view(n, c, -1)
    return (x @ x.transpose(1,2))/(c*h*w)

In [ ]:
#export
#TODO: hardcoded model
feat_m = vgg19(True).features.cuda().eval()
for p in feat_m.parameters(): p.requires_grad=False

In [ ]:
#export
def VggFeats(layers):
  hooks = hook_outputs(layers, detach=False)
  def _inner(x):
    feat_m(x)
    return hooks.stored
  return _inner

In [ ]:
#export
#TODO: hardcoded layers
# layers = [feat_m[i] for i in [1, 11, 18, 25, 20]]; layers # vgg16
layers = [feat_m[i] for i in [1, 6, 11, 20, 29, 22]]; layers # vgg19
get_fs = VggFeats(layers)

In [ ]:
#export
#TODO: hardcoded layers
def get_stl_fs(fs): return fs[:-1]
def get_cnt_fs(fs): return fs[-1:]

In [ ]:
#export
def cnt_loss_fn(fs, y_fs):
  return sum([F.mse_loss(*o) for o in zip(*map(get_cnt_fs, [fs,y_fs]))])

In [ ]:
#export
#TODO: hardcoded stl_loss_mult
def get_style_loss_fn(style_tims):
  stl_fs = [get_fs(im) for im in style_tims]
  # TODO: Weighted mean
  stl_grams = [torch.stack([gram(x) for x in stl_t]).mean(axis=0) for stl_t in zip(*stl_fs)]
  def stl_loss_fn(fs, y_fs):
    bs = fs[0].shape[0]
    stl_losses = [F.mse_loss(y_gram.repeat(bs,1,1),gram(f)) for y_gram,f in zip(*map(get_stl_fs, [stl_grams, fs]))]
    return 3e5*sum(stl_losses)
  return stl_loss_fn

In [ ]:
#export
#TODO: hardcoded tv_loss_mult
def tv_loss_fn(img):      
  tv_h = ((img[:,:,1:,:] - img[:,:,:-1,:]).pow(2)).mean()
  tv_w = ((img[:,:,:,1:] - img[:,:,:,:-1]).pow(2)).mean()    
  return 20*(tv_h + tv_w)

In [ ]:
#export
class FSTLoss(Module):
  def __init__(self, get_fs, stl_loss_fn, cnt_loss_fn, tv_loss_fn):
    super().__init__()
    store_attr(self, 'get_fs,stl_loss_fn,cnt_loss_fn,tv_loss_fn')
    self.metric_names = ['stl', 'cnt', 'tv']

  def forward(self, pred, targ, **kwargs):
    pred_feats,targ_feats = self.get_fs(pred),self.get_fs(targ)
    assert not (pred_feats[0] == targ_feats[0]).all()
    self.stl = self.stl_loss_fn(pred_feats, targ_feats)
    self.cnt = self.cnt_loss_fn(pred_feats, targ_feats)
    self.tv  = self.tv_loss_fn (pred)
    return self.stl+self.cnt+self.tv

In [ ]:
#export
def get_loss_func(style_tims):
  stl_loss_fn = get_style_loss_fn(style_tims)
  return FSTLoss(get_fs, stl_loss_fn, cnt_loss_fn, tv_loss_fn)

## Export -

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.ipynb.
Converted 02_loss.ipynb.
Converted 03_models.ipynb.
Converted Untitled.ipynb.
Converted Untitled1.ipynb.
